## Imports

In [1]:
import os
import json
import shutil
import logging
import datetime
import urlparse
import pandas as pd
import apache_beam as beam
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

from apache_beam.io import textio
from apache_beam.io import tfrecordio

from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3
from tensorflow.contrib.slim.python.slim.nets.inception_v3 import inception_v3_arg_scope
from tensorflow.python.lib.io import file_io
from tensorflow_transform.saved import input_fn_maker
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.beam import impl as beam_impl
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders.csv_coder import CsvCoder
from tensorflow_transform.coders.example_proto_coder import ExampleProtoCoder
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import metadata_io
from tensorflow_transform import coders as tft_coders
from tensorflow_model_analysis.slicer import slicer

from ipywidgets.embed import embed_data

DATA_DIR = 'data/'

/Users/StefanoFiora/.pyenv/versions/taxicab0.6/lib/python2.7/site-packages/pandas/__init__.py:25: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas import hashtable, tslib, lib
/Users/StefanoFiora/.pyenv/versions/taxicab0.6/lib/python2.7/site-packages/pandas/__init__.py:25: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from pandas import hashtable, tslib, lib
/Users/StefanoFiora/.pyenv/versions/taxicab0.6/lib/python2.7/site-packages/pandas/core/common.py:13: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas.algos as algos
/Users/StefanoFiora/.pyenv/versions/taxicab0.6/lib/python2.7/site-packages/pandas/core/common.py:13: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  import pandas.algos as algos
/Users/StefanoFiora/.pyenv/versions/taxicab0.6/lib/pyt

## Functions

In [2]:
# from training.py and transform.py
def make_tft_input_metadata(schema):
    """Make a TFT Schema object
    In the tft framework, this is where default values are recoreded for training.
    Args:
      schema: schema list of training data.
    Returns:
      TFT metadata object.
    """
    tft_schema = {}

    for col_schema in schema:
        col_type = col_schema['type']
        col_name = col_schema['name']
        if col_type == 'NUMBER':
            tft_schema[col_name] = dataset_schema.ColumnSchema(
                tf.float32, [], dataset_schema.FixedColumnRepresentation(default_value=0.0))
        elif col_type in ['CATEGORY', 'TEXT', 'IMAGE_URL', 'KEY']:
            tft_schema[col_name] = dataset_schema.ColumnSchema(
                tf.string, [], dataset_schema.FixedColumnRepresentation(default_value=''))
    return dataset_metadata.DatasetMetadata(dataset_schema.Schema(tft_schema))

# from processing.py
# Categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [24, 31, 12]
CATEGORICAL_FEATURE_KEYS = [
    'trip_start_hour',
    'trip_start_day',
    'trip_start_month'
]

DENSE_FLOAT_FEATURE_KEYS = [
    'trip_miles',
    'fare',
    'trip_seconds'
]

# Number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = [
    'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    'dropoff_longitude'
]

# Number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = [
    'pickup_census_tract',
    'dropoff_census_tract',
    'payment_type',
    'company',
    'pickup_community_area',
    'dropoff_community_area'
]
LABEL_KEY = 'tips'
FARE_KEY = 'fare'


def preprocess(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in DENSE_FLOAT_FEATURE_KEYS:
        # Preserve this feature as a dense float, setting nan's to the mean.
        outputs[key] = tft.scale_to_z_score(inputs[key])

    for key in VOCAB_FEATURE_KEYS:
        # Build a vocabulary for this feature.
        if inputs[key].dtype == tf.string:
            vocab_tensor = inputs[key]
        else:
            vocab_tensor = tf.as_string(inputs[key])
        outputs[key] = tft.string_to_int(
            vocab_tensor, vocab_filename='vocab_' + key,
            top_k=VOCAB_SIZE, num_oov_buckets=OOV_SIZE)

    for key in BUCKET_FEATURE_KEYS:
        outputs[key] = tft.bucketize(inputs[key], FEATURE_BUCKET_COUNT)

    for key in CATEGORICAL_FEATURE_KEYS:
        outputs[key] = tf.to_int64(inputs[key])

    taxi_fare = inputs[FARE_KEY]
    taxi_tip = inputs[LABEL_KEY]
    # Test if the tip was > 20% of the fare.
    tip_threshold = tf.multiply(taxi_fare, tf.constant(0.2))
    outputs[LABEL_KEY] = tf.logical_and(
        tf.logical_not(tf.is_nan(taxi_fare)),
        tf.greater(taxi_tip, tip_threshold))

    return outputs

def get_feature_columns(transformed_data_dir):
    """Callback that returns a list of feature columns for building a tf.estimator.
    Args:
      transformed_data_dir: The GCS directory holding the output of the tft transformation.
    Returns:
      A list of tf.feature_column.
    """
    return (
            [tf.feature_column.numeric_column(key, shape=()) for key in DENSE_FLOAT_FEATURE_KEYS] +
            [tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_identity(
                    key, num_buckets=VOCAB_SIZE + OOV_SIZE))
                for key in VOCAB_FEATURE_KEYS] +
            [tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_identity(
                    key, num_buckets=FEATURE_BUCKET_COUNT, default_value=0))
                for key in BUCKET_FEATURE_KEYS] +
            [tf.feature_column.indicator_column(
                tf.feature_column.categorical_column_with_identity(
                    key, num_buckets=num_buckets, default_value=0))
                for key, num_buckets in zip(CATEGORICAL_FEATURE_KEYS, MAX_CATEGORICAL_FEATURE_VALUES)])

## 1. Data Validation Op

Original: `components/dataflow/tfdv`

In [3]:
# pass

## 2. Transform Op

`components/dataflow/tfft`

In [6]:
# Inception Checkpoint
INCEPTION_V3_CHECKPOINT = 'gs://cloud-ml-data/img/flower_photos/inception_v3_2016_08_28.ckpt'
INCEPTION_EXCLUDED_VARIABLES = ['InceptionV3/AuxLogits', 'InceptionV3/Logits', 'global_step']

DELIMITERS = '.,!?() '


def _image_to_vec(image_str_tensor):
    def _decode_and_resize(image_str_tensor):
        """Decodes jpeg string, resizes it and returns a uint8 tensor."""

        # These constants are set by Inception v3's expectations.
        height = 299
        width = 299
        channels = 3

        image = tf.read_file(image_str_tensor)
        image = tf.image.decode_jpeg(image, channels=channels)
        image = tf.expand_dims(image, 0)
        image = tf.image.resize_bilinear(image, [height, width], align_corners=False)
        image = tf.squeeze(image, squeeze_dims=[0])
        image = tf.cast(image, dtype=tf.uint8)
        return image

    image = tf.map_fn(_decode_and_resize, image_str_tensor, back_prop=False, dtype=tf.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.subtract(image, 0.5)
    inception_input = tf.multiply(image, 2.0)

    # Build Inception layers, which expect a tensor of type float from [-1, 1)
    # and shape [batch_size, height, width, channels].
    with tf.contrib.slim.arg_scope(inception_v3_arg_scope()):
        _, end_points = inception_v3(inception_input, is_training=False)

    embeddings = end_points['PreLogits']
    inception_embeddings = tf.squeeze(embeddings, [1, 2], name='SpatialSqueeze')
    return inception_embeddings


def make_preprocessing_fn(schema):
    """Makes a preprocessing function.
    Args:
      schema: the schema of the training data.
    Returns:
      a preprocessing_fn function used by tft.
    """

    def preprocessing_fn(inputs):
        """TFT preprocessing function.
        Args:
          inputs: dictionary of input `tensorflow_transform.Column`.
        Returns:
          A dictionary of `tensorflow_transform.Column` representing the transformed
              columns.
        """

        features_dict = {}
        for col_schema in schema:
            col_name = col_schema['name']
            if col_schema['type'] == 'NUMBER':
                features_dict[col_name] = inputs[col_name]
            elif col_schema['type'] == 'CATEGORY':
                features_dict[col_name] = tft.string_to_int(inputs[col_name],
                                                            vocab_filename='vocab_' + col_name)
            elif col_schema['type'] == 'TEXT':
                tokens = tf.string_split(inputs[col_name], DELIMITERS)
                # TODO: default_value = 0 is wrong. It means OOV gets 0 for their index.
                # But this is to workaround the issue that trainer can use the true vocab
                # size. Otherwise trainer has to use VOCAB_SIZE defined in this file which
                # is too large. I am talking to TFT folks on this. If there is no workaround,
                # user has to provide a vocab_size.
                VOCAB_SIZE = 100000
                indices = tft.string_to_int(tokens,
                                            vocab_filename='vocab_' + col_name,
                                            default_value=0)
                # Add one for the oov bucket created by string_to_int.
                bow_indices, bow_weights = tft.tfidf(indices, VOCAB_SIZE + 1)
                features_dict[col_name + '_indices'] = bow_indices
                features_dict[col_name + '_weights'] = bow_weights
            elif col_schema['type'] == 'IMAGE_URL':
                features_dict[col_name] = tft.apply_function_with_checkpoint(
                    _image_to_vec,
                    [inputs[col_name]],
                    INCEPTION_V3_CHECKPOINT,
                    exclude=INCEPTION_EXCLUDED_VARIABLES)
            elif col_schema['type'] == 'KEY':
                features_dict[col_name] = inputs[col_name]
            else:
                raise ValueError('Invalid schema. Unknown type ' + col_schema['type'])
        return features_dict

    return preprocessing_fn


def run_transform(output_dir, schema, train_data_file, eval_data_file, preprocessing_fn=None):
    """Writes a tft transform fn, and metadata files.
    Args:
      output_dir: output folder
      schema: schema list.
      train_data_file: training data file pattern.
      eval_data_file: eval data file pattern.
      preprocessing_fn: a function used to preprocess the raw data. If not
                        specified, a function will be automatically inferred
                        from the schema.
    """

    tft_input_metadata = make_tft_input_metadata(schema)
    temp_dir = os.path.join(output_dir, 'tmp')
    preprocessing_fn = preprocessing_fn or make_preprocessing_fn(schema)

    runner = 'DirectRunner'
    with beam.Pipeline(runner, options=None) as p:
        with beam_impl.Context(temp_dir=temp_dir):
            names = [x['name'] for x in schema]
            converter = CsvCoder(names, tft_input_metadata.schema)
            train_data = (
                    p
                    | 'ReadTrainData' >> textio.ReadFromText(train_data_file)
                    | 'DecodeTrainData' >> beam.Map(converter.decode))

            train_dataset = (train_data, tft_input_metadata)
            transformed_dataset, transform_fn = (
                    train_dataset | beam_impl.AnalyzeAndTransformDataset(preprocessing_fn))
            transformed_data, transformed_metadata = transformed_dataset

            # Writes transformed_metadata and transfrom_fn folders
            _ = (transform_fn | 'WriteTransformFn' >> transform_fn_io.WriteTransformFn(output_dir))

            # Write the raw_metadata
            metadata_io.write_metadata(
                metadata=tft_input_metadata,
                path=os.path.join(output_dir, 'metadata'))

            _ = transformed_data | 'WriteTrainData' >> tfrecordio.WriteToTFRecord(
                os.path.join(output_dir, 'train'),
                coder=ExampleProtoCoder(transformed_metadata.schema))

            eval_data = (
                    p
                    | 'ReadEvalData' >> textio.ReadFromText(eval_data_file)
                    | 'DecodeEvalData' >> beam.Map(converter.decode))

            eval_dataset = (eval_data, tft_input_metadata)

            transformed_eval_dataset = (
                    (eval_dataset, transform_fn) | beam_impl.TransformDataset())
            transformed_eval_data, transformed_metadata = transformed_eval_dataset

            _ = transformed_eval_data | 'WriteEvalData' >> tfrecordio.WriteToTFRecord(
                os.path.join(output_dir, 'eval'),
                coder=ExampleProtoCoder(transformed_metadata.schema))

In [8]:
TRAIN_DATA = 'taxi-cab-classification/train.csv'
EVALUATION_DATA = 'taxi-cab-classification/eval.csv'

if os.path.exists(os.path.join(DATA_DIR, "transformed")):
    shutil.rmtree(os.path.join(DATA_DIR, "transformed"))

logging.getLogger().setLevel(logging.INFO)
schema = json.loads(file_io.read_file_to_string(os.path.join(DATA_DIR, "taxi-cab-classification/schema.json")))

def wrapped_preprocessing_fn(inputs):
    outputs = preprocess(inputs)
    for key in outputs:
        if outputs[key].dtype == tf.bool:
            outputs[key] = tft.string_to_int(tf.as_string(outputs[key]),
                                             vocab_filename='vocab_' + key)
    return outputs
preprocessing_fn = wrapped_preprocessing_fn

run_transform(os.path.join(DATA_DIR, "transformed"),
              schema,
              DATA_DIR + TRAIN_DATA,
              DATA_DIR + EVALUATION_DATA,
              preprocessing_fn=preprocessing_fn)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: data/transformed/tmp/tftransform_tmp/c2d261d69f554bf58e772ebc275e5a2b/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: data/transformed/tmp/tftransform_tmp/be3927145796455f9b91ed4d4b53f775/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: data/transformed/tmp/tftransform_tmp/01b393e9db234d7e8f58705584319c2d/saved_model.pb


INFO:root:==================== <function annotate_downstream_side_inputs at 0x123951a28> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x1239518c0> ====================
INFO:root:==================== <function lift_combiners at 0x123951938> ====================
INFO:root:==================== <function expand_gbk at 0x1239519b0> ====================
INFO:root:==================== <function sink_flattens at 0x123951d70> ====================
INFO:root:==================== <function greedily_fuse at 0x123951e60> ====================
INFO:root:==================== <function sort_stages at 0x123951aa0> ====================
INFO:root:Running (ref_AppliedPTransform_AnalyzeAndTransformDataset/AnalyzeDataset/CreateSavedModelForAnalyzerInputs[0]/CreateTransformFn/Read_9)+(ref_PCollection_PCollection_3/Write)
INFO:root:start <DataOutputOperation ref_PCollection_PCollection_3/Write >
INFO:root:start <ReadOperation AnalyzeAndTransformDataset/AnalyzeData

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:root:finish <ReadOperation ReadTrainData/Read source=SourceBundle(weight=1.0, source=<apache_beam.io.textio._TextSource object at 0x1244ad390>, start_position=None, stop_position=None), receivers=[ConsumerSet[ReadTrainData/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation DecodeTrainData output_tags=['out'], receivers=[ConsumerSet[DecodeTrainData.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=3]]>
INFO:root:finish <DoOperation AnalyzeAndTransformDataset/AnalyzeDataset/BatchAnalyzerInputs[0]/BatchElements/ParDo(_GlobalWindowsBatchingDoFn) output_tags=['out'], receivers=[ConsumerSet[AnalyzeAndTransformDataset/AnalyzeDataset/BatchAnalyzerInputs[0]/BatchElements/ParDo(_GlobalWindowsBatchingDoFn).out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation AnalyzeAndTransformData

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

value: "\n\014\n\nConst_19:0\022\nvocab_tips"



value: "\n\014\n\nConst_21:0\022\031vocab_pickup_census_tract"



value: "\n\014\n\nConst_21:0\022\031vocab_pickup_census_tract"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:root:finish <DataInputOperation ref_PCollection_PCollection_466/Read receivers=[ConsumerSet[ref_PCollection_PCollection_466/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation AnalyzeAndTransformDataset/TransformDataset/Transform output_tags=['out'], receivers=[ConsumerSet[AnalyzeAndTransformDataset/TransformDataset/Transform.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation AnalyzeAndTransformDataset/TransformDataset/ConvertAndUnbatch output_tags=['out'], receivers=[ConsumerSet[AnalyzeAndTransformDataset/TransformDataset/ConvertAndUnbatch.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=2]]>
INFO:root:finish <DoOperation WriteTrainData/Write/WriteImpl/WriteBundles output_tags=['out'], receivers=[ConsumerSet[WriteTrainData/Write/WriteImpl/WriteBundles.out0, coder=Wind

## 3. Train Op

`components/kubeflow/datatrainer`

In [9]:
LEARNING_RATE = 0.1
HIDDEN_LAYER_SIZE = '1500'
STEPS = 3
CLASSIFICATION_TARGET_TYPES = [tf.bool, tf.int32, tf.int64]
REGRESSION_TARGET_TYPES = [tf.float32, tf.float64]
TARGET_TYPES = CLASSIFICATION_TARGET_TYPES + REGRESSION_TARGET_TYPES


def is_classification(transformed_data_dir, target):
    """Whether the scenario is classification (vs regression).

    Returns:
      The number of classes if the target represents a classification
      problem, or None if it does not.
    """
    transformed_metadata = metadata_io.read_metadata(
        os.path.join(transformed_data_dir, transform_fn_io.TRANSFORMED_METADATA_DIR))
    transformed_feature_spec = transformed_metadata.schema.as_feature_spec()
    if target not in transformed_feature_spec:
        raise ValueError('Cannot find target "%s" in transformed data.' % target)

    feature = transformed_feature_spec[target]
    if (not isinstance(feature, tf.FixedLenFeature) or feature.shape != [] or
            feature.dtype not in TARGET_TYPES):
        raise ValueError('target "%s" is of invalid type.' % target)

    if feature.dtype in CLASSIFICATION_TARGET_TYPES:
        if feature.dtype == tf.bool:
            return 2
        return get_vocab_size(transformed_data_dir, target)

    return None


def make_training_input_fn(transformed_data_dir, mode, batch_size, target_name, num_epochs=None):
    """Creates an input function reading from transformed data.
    Args:
      transformed_data_dir: Directory to read transformed data and metadata from.
      mode: 'train' or 'eval'.
      batch_size: Batch size.
      target_name: name of the target column.
      num_epochs: number of training data epochs.
    Returns:
      The input function for training or eval.
    """
    transformed_metadata = metadata_io.read_metadata(
        os.path.join(transformed_data_dir, transform_fn_io.TRANSFORMED_METADATA_DIR))
    transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

    def _input_fn():
        """Input function for training and eval."""
        epochs = 1 if mode == 'eval' else num_epochs
        transformed_features = tf.contrib.learn.io.read_batch_features(
            os.path.join(transformed_data_dir, mode + '-*'),
            batch_size, transformed_feature_spec, tf.TFRecordReader, num_epochs=epochs)

        # Extract features and label from the transformed tensors.
        transformed_labels = transformed_features.pop(target_name)
        return transformed_features, transformed_labels

    return _input_fn


def make_serving_input_fn(transformed_data_dir, schema, target_name):
    """Creates an input function reading from transformed data.
    Args:
      transformed_data_dir: Directory to read transformed data and metadata from.
      schema: the raw data schema.
      target_name: name of the target column.
    Returns:
      The input function for serving.
    """
    raw_metadata = make_tft_input_metadata(schema)
    raw_feature_spec = raw_metadata.schema.as_feature_spec()

    raw_keys = [x['name'] for x in schema]
    raw_keys.remove(target_name)
    serving_input_fn = input_fn_maker.build_csv_transforming_serving_input_receiver_fn(
        raw_metadata=raw_metadata,
        transform_savedmodel_dir=transformed_data_dir + '/transform_fn',
        raw_keys=raw_keys)

    return serving_input_fn


def get_vocab_size(transformed_data_dir, feature_name):
    """Get vocab size of a given text or category column."""
    vocab_file = os.path.join(transformed_data_dir,
                              transform_fn_io.TRANSFORM_FN_DIR,
                              'assets',
                              'vocab_' + feature_name)
    with file_io.FileIO(vocab_file, 'r') as f:
        return sum(1 for _ in f)


def get_estimator(schema, transformed_data_dir, target_name, output_dir, hidden_units,
                  optimizer, learning_rate, feature_columns):
    """Get proper tf.estimator (DNNClassifier or DNNRegressor)."""
    optimizer = tf.train.AdagradOptimizer(learning_rate)
    if optimizer == 'Adam':
        optimizer = tf.train.AdamOptimizer(learning_rate)
    elif optimizer == 'SGD':
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)

    # Set how often to run checkpointing in terms of steps.
    config = tf.contrib.learn.RunConfig(save_checkpoints_steps=1000)
    n_classes = is_classification(transformed_data_dir, target_name)
    if n_classes:
        estimator = tf.estimator.DNNClassifier(
            feature_columns=feature_columns,
            hidden_units=hidden_units,
            n_classes=n_classes,
            config=config,
            model_dir=output_dir)
    else:
        estimator = tf.estimator.DNNRegressor(
            feature_columns=feature_columns,
            hidden_units=hidden_units,
            config=config,
            model_dir=output_dir,
            optimizer=optimizer)

    return estimator


def eval_input_receiver_fn(tf_transform_dir, schema, target):
    """Build everything needed for the tf-model-analysis to run the model.
    Args:
      tf_transform_dir: directory in which the tf-transform model was written
        during the preprocessing step.
      schema: the raw data schema.
      target: name of the target column.
    Returns:
      EvalInputReceiver function, which contains:
        - Tensorflow graph which parses raw untranformed features, applies the
          tf-transform preprocessing operators.
        - Set of raw, untransformed features.
        - Label against which predictions will be compared.
    """
    raw_metadata = make_tft_input_metadata(schema)
    raw_feature_spec = raw_metadata.schema.as_feature_spec()
    serialized_tf_example = tf.placeholder(
        dtype=tf.string, shape=[None], name='input_example_tensor')
    features = tf.parse_example(serialized_tf_example, raw_feature_spec)
    _, transformed_features = (
        saved_transform_io.partially_apply_saved_transform(
            os.path.join(tf_transform_dir, transform_fn_io.TRANSFORM_FN_DIR),
            features))
    receiver_tensors = {'examples': serialized_tf_example}
    return tfma.export.EvalInputReceiver(
        features=transformed_features,
        receiver_tensors=receiver_tensors,
        labels=transformed_features[target])

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)

hidden_layer_size = [int(x.strip()) for x in HIDDEN_LAYER_SIZE.split(',')]
schema = json.loads(file_io.read_file_to_string(os.path.join(DATA_DIR, "taxi-cab-classification/schema.json")))
feature_columns = None
feature_columns = get_feature_columns(os.path.join(DATA_DIR, "transformed"))
estimator = get_estimator(schema, 
                          os.path.join(DATA_DIR, "transformed"), 
                          "tips",
                          os.path.join(DATA_DIR, "training"),
                          hidden_layer_size,
                          "Adagrad", 
                          LEARNING_RATE,
                          feature_columns)

# TODO: Expose batch size.
train_input_fn = make_training_input_fn(
    os.path.join(DATA_DIR, "transformed"),
    'train',
    32,
    "tips",
    num_epochs=2)

eval_input_fn = make_training_input_fn(
    os.path.join(DATA_DIR, "transformed"),
    'eval',
    32,
    "tips")
serving_input_fn = make_serving_input_fn(
    os.path.join(DATA_DIR, "transformed"),
    schema,
    "tips")

exporter = tf.estimator.FinalExporter('export', serving_input_fn)
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=STEPS)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, exporters=[exporter])
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

eval_model_dir = os.path.join(DATA_DIR + "training", 'tfma_eval_model_dir')
tfma.export.export_eval_savedmodel(
    estimator=estimator,
    export_dir_base=eval_model_dir,
    eval_input_receiver_fn=(
        lambda: eval_input_receiver_fn(
            os.path.join(DATA_DIR, "transformed"), schema, "tips")))

metadata = {
    'outputs': [{
        'type': 'tensorboard',
        'source': os.path.join(DATA_DIR, "training"),
    }]
}

try:
    os.makedirs(os.path.join(DATA_DIR, "artifacts", "training"))
except OSError:
    pass
with open(os.path.join(DATA_DIR, "artifacts", "training", 'mlpipeline-ui-metadata.json'), 'w') as f:
    json.dump(metadata, f)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125af4ed0>, '_model_dir': 'data/training', '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}


INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x125af4ed0>, '_model_dir': 'data/training', '_save_checkpoints_steps': 1000, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.


...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets written to: data/training/tfma_eval_model_dir/temp-1567670382/assets


INFO:tensorflow:Assets written to: data/training/tfma_eval_model_dir/temp-1567670382/assets


INFO:tensorflow:SavedModel written to: data/training/tfma_eval_model_dir/temp-1567670382/saved_model.pb


INFO:tensorflow:SavedModel written to: data/training/tfma_eval_model_dir/temp-1567670382/saved_model.pb


## 4. Analyze Op

`components/dataflow/tfma`

In [17]:
_OUTPUT_HTML_FILE = 'output_display.html'
_STATIC_HTML_TEMPLATE = """
<html>
  <head>
    <title>TFMA Slicing Metrics</title>

    <!-- Load RequireJS, used by the IPywidgets for dependency management -->
    <script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js"
            integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA="
            crossorigin="anonymous">
    </script>

    <!-- Load IPywidgets bundle for embedding. -->
    <script src="https://unpkg.com/@jupyter-widgets/html-manager@^0.12.0/dist/embed-amd.js"
            crossorigin="anonymous">
    </script>

    <!-- Load IPywidgets bundle for embedding. -->
    <script>
      require.config({{
        paths: {{
          "tfma_widget_js": "https://cdn.rawgit.com/tensorflow/model-analysis/v0.6.0/tensorflow_model_analysis/static/index"
        }}
      }});
    </script>

    <link rel="import" href="https://cdn.rawgit.com/tensorflow/model-analysis/v0.6.0/tensorflow_model_analysis/static/vulcanized_template.html">

    <!-- The state of all the widget models on the page -->
    <script type="application/vnd.jupyter.widget-state+json">
      {manager_state}
    </script>
  </head>

  <body>
    <h1>TFMA Slicing Metrics</h1>
    {widget_views}
  </body>
</html>
"""
_SINGLE_WIDGET_TEMPLATE = """
    <div id="slicing-metrics-widget-{0}">
      <script type="application/vnd.jupyter.widget-view+json">
        {1}
      </script>
    </div>
"""


def get_raw_feature_spec(schema):
    feature_spec = {}
    for column in schema:
        column_name = column['name']
        column_type = column['type']

        feature = tf.FixedLenFeature(shape=[], dtype=tf.string, default_value='')
        if column_type == 'NUMBER':
            feature = tf.FixedLenFeature(shape=[], dtype=tf.float32, default_value=0.0)
        feature_spec[column_name] = feature
    return feature_spec


def clean_raw_data_dict(raw_feature_spec):
    def clean_method(input_dict):
        output_dict = {}

        for key in raw_feature_spec:
            if key not in input_dict or not input_dict[key]:
                output_dict[key] = raw_feature_spec[key].default_value
            else:
                output_dict[key] = input_dict[key]
        return output_dict

    return clean_method


def run_analysis(output_dir, model_dir, eval_path, schema, slice_columns):
    runner = 'DirectRunner'
    column_names = [x['name'] for x in schema]
    for slice_column in slice_columns:
        if slice_column not in column_names:
            raise ValueError("Unknown slice column: %s" % slice_column)

    slice_spec = [
        slicer.SingleSliceSpec(),  # An empty spec is required for the 'Overall' slice
        slicer.SingleSliceSpec(columns=slice_columns)
    ]

    with beam.Pipeline(runner=runner, options=None) as pipeline:
        raw_feature_spec = get_raw_feature_spec(schema)
        raw_schema = dataset_schema.from_feature_spec(raw_feature_spec)
        example_coder = tft_coders.example_proto_coder.ExampleProtoCoder(raw_schema)
        csv_coder = tft_coders.CsvCoder(column_names, raw_schema)

        raw_data = (
                pipeline
                | 'ReadFromText' >> beam.io.ReadFromText(eval_path)
                | 'ParseCSV' >> beam.Map(csv_coder.decode)
                | 'CleanData' >> beam.Map(clean_raw_data_dict(raw_feature_spec))
                | 'ToSerializedTFExample' >> beam.Map(example_coder.encode)
                | 'EvaluateAndWriteResults' >> tfma.EvaluateAndWriteResults(
            eval_saved_model_path=model_dir,
            slice_spec=slice_spec,
            output_path=output_dir))


def generate_static_html_output(output_dir, slicing_columns):
    result = tfma.load_eval_result(output_path=output_dir)
    slicing_metrics_views = [
        tfma.view.render_slicing_metrics(result, slicing_column=slicing_column)
        for slicing_column in slicing_columns
    ]
    data = embed_data(views=slicing_metrics_views)
    manager_state = json.dumps(data['manager_state'])
    widget_views = [json.dumps(view) for view in data['view_specs']]
    views_html = ""
    for idx, view in enumerate(widget_views):
        views_html += _SINGLE_WIDGET_TEMPLATE.format(idx, view)
    rendered_template = _STATIC_HTML_TEMPLATE.format(
        manager_state=manager_state, widget_views=views_html)
    static_html_path = os.path.join(output_dir, _OUTPUT_HTML_FILE)
    file_io.write_string_to_file(static_html_path, rendered_template)

    metadata = {
        'outputs': [{
            'type': 'web-app',
            'storage': 'gcs',
            'source': static_html_path,
        }]
    }
    try:
        os.makedirs(os.path.join(DATA_DIR, "artifacts", "analyze"))
    except OSError:
        pass
    with file_io.FileIO(os.path.join(DATA_DIR, 'artifacts', 'analyze', 'mlpipeline-ui-metadata.json'), 'w') as f:
        json.dump(metadata, f)

In [18]:
SLICE_COLUMN = ['trip_start_hour']

tf.logging.set_verbosity(tf.logging.INFO)

slice_columns = [
    column
    for column_group in SLICE_COLUMN
    for column in column_group.split(',')
]
schema = json.loads(file_io.read_file_to_string(os.path.join(DATA_DIR, "taxi-cab-classification/schema.json")))
eval_model_parent_dir = os.path.join(DATA_DIR, "training", 'tfma_eval_model_dir')
model_export_dir = os.path.join(eval_model_parent_dir, file_io.list_directory(eval_model_parent_dir)[0])
run_analysis(os.path.join(DATA_DIR, "analysis"),
             model_export_dir,
             DATA_DIR + EVALUATION_DATA,
             schema,
             slice_columns)
generate_static_html_output(os.path.join(DATA_DIR, "analysis"), slice_columns)

INFO:root:==================== <function annotate_downstream_side_inputs at 0x1258207d0> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x125820e60> ====================
INFO:root:==================== <function lift_combiners at 0x125820320> ====================
INFO:root:==================== <function expand_gbk at 0x125820668> ====================
INFO:root:==================== <function sink_flattens at 0x1258208c0> ====================
INFO:root:==================== <function greedily_fuse at 0x125820a28> ====================
INFO:root:==================== <function sort_stages at 0x125820b90> ====================
INFO:root:Running ((ref_AppliedPTransform_ReadFromText/Read_3)+((ref_AppliedPTransform_ParseCSV_4)+((ref_AppliedPTransform_CleanData_5)+((ref_AppliedPTransform_ToSerializedTFExample_6)+((ref_AppliedPTransform_EvaluateAndWriteResults/Evaluate/Predict/Batch/ParDo(_GlobalWindowsBatchingDoFn)_11)+((ref_AppliedPTransform_EvaluateAn

INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables
INFO:root:start <DoOperation EvaluateAndWriteResults/Evaluate/Predict/Batch/ParDo(_GlobalWindowsBatchingDoFn) output_tags=['out']>
INFO:root:start <DoOperation ToSerializedTFExample output_tags=['out']>
INFO:root:start <DoOperation CleanData output_tags=['out']>
INFO:root:start <DoOperation ParseCSV output_tags=['out']>
INFO:root:start <ReadOperation ReadFromText/Read source=SourceBundle(weight=1.0, source=<apache_beam.io.textio._TextSource object at 0x1256e0910>, start_position=None, stop_position=None)>


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables
INFO:root:finish <ReadOperation ReadFromText/Read source=SourceBundle(weight=1.0, source=<apache_beam.io.textio._TextSource object at 0x1256e0910>, start_position=None, stop_position=None), receivers=[ConsumerSet[ReadFromText/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation ParseCSV output_tags=['out'], receivers=[ConsumerSet[ParseCSV.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation CleanData output_tags=['out'], receivers=[ConsumerSet[CleanData.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation ToSerializedTFExample output_tags=['out'], receivers=[ConsumerSet[ToSerializedTFExample.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation EvaluateAndWriteResults/Evaluate/Pre

INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables
INFO:root:start <CombineOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/ExtractOutputs phase=extract>
INFO:root:start <CombineOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Merge phase=merge>
INFO:root:start <DataInputOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Group/Read receivers=[ConsumerSet[EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Group/Read.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], IterableCoder[LengthPrefixCoder[FastPrimitivesCoder]]]], len(consumers)=1]]>


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables


INFO:tensorflow:Restoring parameters from data/training/tfma_eval_model_dir/1567670214/variables/variables
INFO:root:finish <DataInputOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Group/Read receivers=[ConsumerSet[EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Group/Read.out0, coder=WindowedValueCoder[TupleCoder[LengthPrefixCoder[FastPrimitivesCoder], IterableCoder[LengthPrefixCoder[FastPrimitivesCoder]]]], len(consumers)=1]]>
INFO:root:finish <CombineOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Merge phase=merge, receivers=[ConsumerSet[EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/Merge.out0, coder=WindowedValueCoder[TupleCoder[FastPrimitivesCoder, FastPrimitivesCoder]], len(consumers)=1]]>
INFO:root:finish <CombineOperation EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/ExtractOutputs phase=extract, receivers=[ConsumerSet[EvaluateAndWriteResults/Evaluate/Aggregate/CombinePerKey/ExtractOutputs.out0, coder=Window

## 5. Predict Op

`components/dataflow/predict`

In [19]:
class EmitAsBatchDoFn(beam.DoFn):
    """A DoFn that buffers the records and emits them batch by batch."""

    def __init__(self, batch_size):
        self._batch_size = batch_size
        self._cached = []

    def process(self, element):
        from apache_beam.transforms import window
        from apache_beam.utils.windowed_value import WindowedValue
        self._cached.append(element)
        if len(self._cached) >= self._batch_size:
            emit = self._cached
            self._cached = []
            yield emit

    def finish_bundle(self, context=None):
        from apache_beam.transforms import window
        from apache_beam.utils.windowed_value import WindowedValue
        if len(self._cached) > 0:
            yield WindowedValue(self._cached, -1, [window.GlobalWindow()])


class TargetToLastDoFn(beam.DoFn):
    """A DoFn that moves specified target column to last."""

    def __init__(self, names, target_name):
        self._names = names
        self._target_name = target_name
        self._names_no_target = list(names)
        self._names_no_target.remove(target_name)

    def process(self, element):
        import csv
        content = csv.DictReader([element], fieldnames=self._names).next()
        target = content.pop(self._target_name)
        yield [content[x] for x in self._names_no_target] + [target]


class PredictDoFn(beam.DoFn):
    """A DoFn that performs predictions with given trained model."""

    def __init__(self, model_export_dir):
        self._model_export_dir = model_export_dir

    def start_bundle(self):
        from tensorflow.contrib import predictor

        # We need to import the tensorflow_transform library in order to
        # register all of the ops that might be used by a saved model that
        # incorporates TFT transformations.
        import tensorflow_transform

        self._predict_fn = predictor.from_saved_model(self._model_export_dir)

    def process(self, element):
        import csv
        import StringIO

        prediction_inputs = []
        for instance in element:
            instance_copy = list(instance)
            instance_copy.pop()  # remove target
            buf = StringIO.StringIO()
            writer = csv.writer(buf, lineterminator='')
            writer.writerow(instance_copy)
            prediction_inputs.append(buf.getvalue())

        return_dict = self._predict_fn({"inputs": prediction_inputs})
        return_dict['source'] = element
        yield return_dict


class ListToCsvDoFn(beam.DoFn):
    """A DoFn function that convert list to csv line."""

    def process(self, element):
        import csv
        import StringIO
        buf = StringIO.StringIO()
        writer = csv.writer(buf, lineterminator='')
        writer.writerow(element)
        yield buf.getvalue()

In [20]:
def run_predict(output_dir, data_path, schema, target_name, model_export_dir, batch_size):
    """Run predictions with given model using DataFlow.
    Args:
      output_dir: output folder
      data_path: test data file path.
      schema: schema list.
      target_name: target column name.
      model_export_dir: GCS or local path of exported model trained with tft preprocessed data.
      batch_size: batch size when running prediction.
    """

    target_type = next(x for x in schema if x['name'] == target_name)['type']
    labels_file = os.path.join(model_export_dir, 'assets', 'vocab_' + target_name)
    is_classification = file_io.file_exists(labels_file)

    output_file_prefix = os.path.join(output_dir, 'prediction_results')
    output_schema_file = os.path.join(output_dir, 'schema.json')
    names = [x['name'] for x in schema]

    output_schema = filter(lambda x: x['name'] != target_name, schema)
    if is_classification:
        with file_io.FileIO(labels_file, mode='r') as f:
            labels = [x.strip() for x in f.readlines()]

        output_schema.append({'name': 'target', 'type': 'CATEGORY'})
        output_schema.append({'name': 'predicted', 'type': 'CATEGORY'})
        output_schema.extend([{'name': x, 'type': 'NUMBER'} for x in labels])
    else:
        output_schema.append({'name': 'target', 'type': 'NUMBER'})
        output_schema.append({'name': 'predicted', 'type': 'NUMBER'})

    runner = 'DirectRunner'
    with beam.Pipeline(runner, options=None) as p:
        raw_results = (p
                       | 'read data' >> beam.io.ReadFromText(data_path)
                       | 'move target to last' >> beam.ParDo(TargetToLastDoFn(names, target_name))
                       | 'batch' >> beam.ParDo(EmitAsBatchDoFn(batch_size))
                       | 'predict' >> beam.ParDo(PredictDoFn(model_export_dir)))

        if is_classification:
            processed_results = (raw_results
                                 | 'unbatch' >> beam.FlatMap(lambda x: zip(x['source'], x['scores']))
                                 | 'get predicted' >> beam.Map(lambda x: x[0] + [labels[x[1].argmax()]] + list(x[1])))
        else:
            processed_results = (raw_results
                                 | 'unbatch' >> beam.FlatMap(lambda x: zip(x['source'], x['outputs']))
                                 | 'get predicted' >> beam.Map(lambda x: x[0] + list(x[1])))

        results_save = (processed_results
                        | 'write csv lines' >> beam.ParDo(ListToCsvDoFn())
                        | 'write file' >> beam.io.WriteToText(output_file_prefix))

        (results_save
         | 'fixed one' >> beam.transforms.combiners.Sample.FixedSizeGlobally(1)
         | 'set schema' >> beam.Map(lambda path: json.dumps(output_schema))
         | 'write schema file' >> beam.io.WriteToText(output_schema_file, shard_name_template=''))


logging.getLogger().setLevel(logging.INFO)

# Models trained with estimator are exported to base/export/export/123456781 directory.
# Our trainer export only one model.
model = os.path.join(DATA_DIR, 'training')  # args.model
export_parent_dir = os.path.join(model, 'export', 'export')
model_export_dir = os.path.join(export_parent_dir, file_io.list_directory(export_parent_dir)[0])
schema = json.loads(file_io.read_file_to_string(os.path.join(DATA_DIR, "taxi-cab-classification/schema.json")))

batchsize = 32
run_predict(os.path.join(DATA_DIR, "predict"),
            os.path.join(DATA_DIR, EVALUATION_DATA),
            schema,
            "tips",
            model_export_dir,
            batchsize)
prediction_results = os.path.join(DATA_DIR + "predict", 'prediction_results-*')

with file_io.FileIO(os.path.join(DATA_DIR, "predict", 'schema.json'), 'r') as f:
    schema = json.load(f)

metadata = {
    'outputs': [{
        'type': 'table',
        'storage': 'gcs',
        'format': 'csv',
        'header': [x['name'] for x in schema],
        'source': prediction_results
    }]
}
try:
    os.makedirs(os.path.join(DATA_DIR, "artifacts", "predict"))
except OSError:
    pass
with open(os.path.join(DATA_DIR, 'artifacts', 'predict', 'mlpipeline-ui-metadata.json'), 'w') as f:
    json.dump(metadata, f)

INFO:root:==================== <function annotate_downstream_side_inputs at 0x1256be320> ====================
INFO:root:==================== <function fix_side_input_pcoll_coders at 0x1256bee60> ====================
INFO:root:==================== <function lift_combiners at 0x12484d6e0> ====================
INFO:root:==================== <function expand_gbk at 0x12484d140> ====================
INFO:root:==================== <function sink_flattens at 0x1256be938> ====================
INFO:root:==================== <function greedily_fuse at 0x1256be668> ====================
INFO:root:==================== <function sort_stages at 0x1256be398> ====================
INFO:root:Running (((ref_AppliedPTransform_write schema file/Write/WriteImpl/DoOnce/Read_45)+(ref_AppliedPTransform_write schema file/Write/WriteImpl/InitializeWrite_46))+(ref_PCollection_PCollection_28/Write))+(ref_PCollection_PCollection_29/Write)
INFO:root:start <DataOutputOperation ref_PCollection_PCollection_29/Write >
IN

INFO:tensorflow:Restoring parameters from data/training/export/export/1567670379/variables/variables


INFO:tensorflow:Restoring parameters from data/training/export/export/1567670379/variables/variables
INFO:root:start <DoOperation batch output_tags=['out']>
INFO:root:start <DoOperation move target to last output_tags=['out']>
INFO:root:start <ReadOperation read data/Read source=SourceBundle(weight=1.0, source=<apache_beam.io.textio._TextSource object at 0x1251bfbd0>, start_position=None, stop_position=None)>
INFO:root:finish <ReadOperation read data/Read source=SourceBundle(weight=1.0, source=<apache_beam.io.textio._TextSource object at 0x1251bfbd0>, start_position=None, stop_position=None), receivers=[ConsumerSet[read data/Read.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation move target to last output_tags=['out'], receivers=[ConsumerSet[move target to last.out0, coder=WindowedValueCoder[FastPrimitivesCoder], len(consumers)=1]]>
INFO:root:finish <DoOperation batch output_tags=['out'], receivers=[ConsumerSet[batch.out0, coder=Wind

## 6. Confusion Matrix Op

`components/local/confusion_matrix`

In [22]:
predictions = os.path.join(DATA_DIR, 'predict', 'prediction_results-*')
target_lambda = """lambda x: (x['target'] > x['fare'] * 0.2)"""
if not os.path.exists(os.path.join(DATA_DIR, "confusionmatrix")):
    os.makedirs(os.path.join(DATA_DIR, "confusionmatrix"))

schema_file = os.path.join(os.path.dirname(predictions), 'schema.json')
schema = json.loads(file_io.read_file_to_string(schema_file))
names = [x['name'] for x in schema]
dfs = []
files = file_io.get_matching_files(predictions)
for file in files:
    with file_io.FileIO(file, 'r') as f:
        dfs.append(pd.read_csv(f, names=names))

df = pd.concat(dfs)
if target_lambda:
    df['target'] = df.apply(eval(target_lambda), axis=1)

vocab = list(df['target'].unique())
cm = confusion_matrix(df['target'], df['predicted'], labels=vocab)
data = []
for target_index, target_row in enumerate(cm):
    for predicted_index, count in enumerate(target_row):
        data.append((vocab[target_index], vocab[predicted_index], count))

df_cm = pd.DataFrame(data, columns=['target', 'predicted', 'count'])
cm_file = os.path.join(DATA_DIR, 'confusionmatrix', 'confusion_matrix.csv')
with file_io.FileIO(cm_file, 'w') as f:
    df_cm.to_csv(f, columns=['target', 'predicted', 'count'], header=False, index=False)

metadata = {
    'outputs': [{
        'type': 'confusion_matrix',
        'format': 'csv',
        'schema': [
            {'name': 'target', 'type': 'CATEGORY'},
            {'name': 'predicted', 'type': 'CATEGORY'},
            {'name': 'count', 'type': 'NUMBER'},
        ],
        'source': cm_file,
        # Convert vocab to string because for bealean values we want "True|False" to match csv data.
        'labels': list(map(str, vocab)),
    }]
}
try:
    os.makedirs(os.path.join(DATA_DIR, "artifacts", "confusionmatrix"))
except OSError:
    pass
with file_io.FileIO(os.path.join(DATA_DIR, 'artifacts', 'confusionmatrix', 'mlpipeline-ui-metadata.json'), 'w') as f:
    json.dump(metadata, f)

accuracy = accuracy_score(df['target'], df['predicted'])
metrics = {
    'metrics': [{
        'name': 'accuracy-score',
        'numberValue': accuracy,
        'format': "PERCENTAGE",
    }]
}
with file_io.FileIO(os.path.join(DATA_DIR, 'artifacts', 'confusionmatrix', 'mlpipeline-metrics.json'), 'w') as f:
    json.dump(metrics, f)

## 7. ROC Op

`components/local/roc`

In [23]:
predictions = os.path.join(DATA_DIR, 'predict', 'prediction_results-*')
target_lambda = """lambda x: 1 if (x['target'] > x['fare'] * 0.2) else 0"""
true_class = 'true'
true_score_column = 'true'

if not os.path.exists(os.path.join(DATA_DIR, "roc")):
    os.makedirs(os.path.join(DATA_DIR, "roc"))

schema_file = os.path.join(os.path.dirname(predictions), 'schema.json')
schema = json.loads(file_io.read_file_to_string(schema_file))
names = [x['name'] for x in schema]

if not target_lambda and 'target' not in names:
    raise ValueError('There is no "target" column, and target_lambda is not provided.')

if true_score_column not in names:
    raise ValueError('Cannot find column name "%s"' % true_score_column)

dfs = []
files = file_io.get_matching_files(predictions)
for file in files:
    with file_io.FileIO(file, 'r') as f:
        dfs.append(pd.read_csv(f, names=names))

df = pd.concat(dfs)
if target_lambda:
    df['target'] = df.apply(eval(target_lambda), axis=1)
else:
    df['target'] = df['target'].apply(lambda x: 1 if x == true_class else 0)
fpr, tpr, thresholds = roc_curve(df['target'], df[true_score_column])
roc_auc = roc_auc_score(df['target'], df[true_score_column])
df_roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr, 'thresholds': thresholds})
roc_file = os.path.join(DATA_DIR + "roc", 'roc.csv')
with file_io.FileIO(roc_file, 'w') as f:
    df_roc.to_csv(f, columns=['fpr', 'tpr', 'thresholds'], header=False, index=False)

metadata = {
    'outputs': [{
        'type': 'roc',
        'format': 'csv',
        'schema': [
            {'name': 'fpr', 'type': 'NUMBER'},
            {'name': 'tpr', 'type': 'NUMBER'},
            {'name': 'thresholds', 'type': 'NUMBER'},
        ],
        'source': roc_file
    }]
}
try:
    os.makedirs(os.path.join(DATA_DIR, "artifacts", "roc"))
except OSError:
    pass
with file_io.FileIO(os.path.join(DATA_DIR, 'artifacts', 'roc', 'mlpipeline-ui-metadata.json'), 'w') as f:
    json.dump(metadata, f)

metrics = {
    'metrics': [{
        'name': 'roc-auc-score',
        'numberValue': roc_auc,
    }]
}
with file_io.FileIO(os.path.join(DATA_DIR, 'artifacts', 'roc', 'mlpipeline-metrics.json'), 'w') as f:
    json.dump(metrics, f)

## 8. Deploy Op

`components/kubeflow/deployer`

In [ ]:
# pass